In [1]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from glob import glob
from tqdm import tqdm
import time
import json
import os
import gc

In [2]:
files = glob('subtopics/*.json')
len(files)

7502

In [3]:
urls = []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)

    for d in data:
        if d[-1] > 1:
            url = d[0]
            for i in range(2, d[-1] + 1, 1):
                urls.append(f'{url}page-{i}')

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 7502/7502 [00:13<00:00, 564.20it/s]


In [4]:
urls = sorted(urls)

In [5]:
len(urls), len(urls) // 50

(1608084, 32161)

In [6]:
import time

def get_href_topic(url):
    while True:
        try:
            r = requests.get(url, timeout = 5.0)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    div = soup.find_all('div', {'class': 'message-userContent lbContainer js-lbContainer'})
    return {'div': [str(d) for d in div], 'url': url}


filtered = get_href_topic(urls[0])

In [7]:
filtered

{'div': ['<div class="message-userContent lbContainer js-lbContainer" data-lb-caption-desc="abeInc · Mar 10, 2012 at 12:55 PM" data-lb-id="post-12517575">\n<article class="message-body js-selectToQuote">\n<div itemprop="text">\n<div class="bbWrapper"><blockquote class="bbCodeBlock bbCodeBlock--expandable bbCodeBlock--quote js-expandWatch" data-attributes="member: 183099" data-quote="aduhaiya" data-source="post: 12515829">\n<div class="bbCodeBlock-title">\n<a class="bbCodeBlock-sourceJump" data-content-selector="#post-12515829" data-xf-click="attribution" href="/forum/goto/post?id=12515829" rel="nofollow">aduhaiya said:</a>\n</div>\n<div class="bbCodeBlock-content">\n<div class="bbCodeBlock-expandContent js-expandContent">\nhmmm... sbnrnya... byk lagi jawatan kosong yg nak diiklankan... tapi... 4 jawatan nie sangat diperlukan segera. Tapi... apa2pun hantar shj resume kpd kami dan bila dah ada jawatan kosong utk IT kami terus hubungi anda. Hantarkan resume anda ke <a href="/cdn-cgi/l/ema

In [8]:
!rm -rf pages-v2
!mkdir pages-v2

In [9]:
max_worker = 50
for i in tqdm(range(0, len(urls), max_worker)):
    
    filename = os.path.join('pages-v2', f'{i}.json')
    if os.path.exists(filename):
        continue
        
    gc.collect()
    
    results = []
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(get_href_topic, url): url for url in urls[i: i + max_worker]}

        for future in as_completed(futures):
            results.append(future.result())
    
    with open(filename, 'w') as fopen:
        json.dump(results, fopen)
        
    try:
        del results
        del futures
    except:
        pass

 15%|███████████████▎                                                                                   | 4958/32162 [6:50:41<34:47:42,  4.60s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

 50%|████████████████████████████████████████████████▌                                                | 16109/32162 [21:58:00<22:05:29,  4.95s/it]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

100%|█████████████████████████████████████████████████████

In [10]:
!du -hs pages

15G	pages


In [12]:
# !zip -r pages.zip pages

In [54]:
files = glob('subtopics/*.json')
len(files)

7502

In [55]:
df = []
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    
    for d in data:
        forum_id = d[0]
        for d_ in d[1]:
            soup = BeautifulSoup(d_, "lxml")
            quote = soup.find('blockquote')
            if quote is not None:
                soup.find('blockquote').decompose()

            text = soup.get_text(separator="\n")
            df.append({
                'forum_id': forum_id,
                'text': text.strip(),
            })

100%|███████████████████████████████████████| 7502/7502 [11:33<00:00, 10.82it/s]


In [56]:
len(df)

2458217

In [57]:
df[:3]

[{'forum_id': 'https://carigold.com/forum/threads/himpunan-post2-member-cg-yg-offtopik-dari-thread2-lain.744033/',
  'text': 'pasaipa jd lagu tuh...? hang usik bini org ka.. guruau bini org boh penampar.. kaco bini org boh penombok la dauz'},
 {'forum_id': 'https://carigold.com/forum/threads/himpunan-post2-member-cg-yg-offtopik-dari-thread2-lain.744033/',
  'text': 'Citer la sikit.'},
 {'forum_id': 'https://carigold.com/forum/threads/himpunan-post2-member-cg-yg-offtopik-dari-thread2-lain.744033/',
  'text': 'Kena masa form 5'}]

In [58]:
files = glob('pages-v2/*.json')
len(files)

32162

In [59]:
with open(files[0]) as fopen:
    data = json.load(fopen)

In [61]:
for f in tqdm(files):
    with open(f) as fopen:
        data = json.load(fopen)
    
    for d in data:
        forum_id = d['url']
        for d_ in d['div']:
            soup = BeautifulSoup(d_, "lxml")
            quote = soup.find('blockquote')
            if quote is not None:
                soup.find('blockquote').decompose()

            text = soup.get_text(separator="\n")
            df.append({
                'forum_id': forum_id,
                'text': text.strip(),
            })

100%|███████████████████████████████████| 32162/32162 [1:22:11<00:00,  6.52it/s]


In [63]:
len(df)

17664954

In [66]:
import pandas as pd
df_pd = pd.DataFrame(df)

In [67]:
df_pd.to_parquet('posts.parquet', index = False)

In [68]:
df_pd.head()

,forum_id,text
0,https://carigold.com/forum/threads/himpunan-po...,pasaipa jd lagu tuh...? hang usik bini org ka....
1,https://carigold.com/forum/threads/himpunan-po...,Citer la sikit.
2,https://carigold.com/forum/threads/himpunan-po...,Kena masa form 5
3,https://carigold.com/forum/threads/himpunan-po...,Ya Allah ya Tuhanku celaka la mulut ko bro fit...
4,https://carigold.com/forum/threads/himpunan-po...,Macam nie la tanya kenapa nie kata saya kacau ...
